In [1]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
PATH = '/content/drive/My Drive/open/'

In [3]:
!pip install autokeras keras-tuner konlpy

     |████████████████████████████████| 174kB 9.4MB/s 
     |████████████████████████████████| 71kB 7.9MB/s 
     |████████████████████████████████| 19.4MB 82.6MB/s 
     |████████████████████████████████| 460kB 53.3MB/s 
     |████████████████████████████████| 92kB 12.8MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=f289bbb20d21589b2422a9804741b2c1c069cd18e8d3bf2ae9d433d6e22229ad
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=eb863ca13dc808a8efea4e3727f42b5eecfa2ae6277a9b23686271c26201b84b
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup

In [4]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh
%cd ..

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 75 (delta 33), reused 20 (delta 5), pack-reused 0
Unpacking objects: 100% (75/75), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2020-12-31 05:24:22--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::6b17:d1f5, 2406:da00:ff00::22c3:9b0a, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?Signature=b4NRoOfQf2XGwmh8OOJZx6MfHoo%3D&Expires=1609393279&AWSAccessKeyId=AKIA6K

In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
import autokeras as ak
from konlpy.tag import Mecab

In [6]:
train = pd.read_csv(PATH + "news_train.csv")
test = pd.read_csv(PATH + "news_test.csv")
submission = pd.read_csv(PATH + "sample_submission.csv")

In [7]:
mecab = Mecab()

In [8]:
train['content'] = train['content'].map(lambda x: ' '.join(mecab.morphs(x)))
test['content'] = test['content'].map(lambda x: ' '.join(mecab.morphs(x)))

In [9]:
x_train = train['content'].values
y_train = train['info'].values

In [10]:
input_node = ak.TextInput()
output_node = ak.TextBlock()(input_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node,
    outputs=output_node,
    overwrite=True,
    max_trials=20)
clf.fit(x_train, y_train, epochs=5)
model = clf.export_model()

Trial 20 Complete [00h 03m 06s]
val_loss: 0.0026727134827524424

Best val_loss So Far: 0.0004164638230577111
Total elapsed time: 01h 05m 27s
INFO:tensorflow:Oracle triggered exit
Epoch 1/5
3711/3711 [==============================] - 43s 11ms/step - loss: 0.1730 - accuracy: 0.9415
Epoch 2/5
3711/3711 [==============================] - 42s 11ms/step - loss: 0.0929 - accuracy: 0.9749
Epoch 3/5
3711/3711 [==============================] - 41s 11ms/step - loss: 0.0562 - accuracy: 0.9843
Epoch 4/5
3711/3711 [==============================] - 41s 11ms/step - loss: 0.0398 - accuracy: 0.9883
Epoch 5/5
3711/3711 [==============================] - 41s 11ms/step - loss: 0.0288 - accuracy: 0.9916
INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


In [11]:
tf.keras.backend.clear_session()
model = tf.keras.models.load_model('./auto_model/best_model')

In [12]:
model.fit(x_train, y_train, epochs=100, validation_split=0.2)

Epoch 1/100
2969/2969 [==============================] - 38s 13ms/step - loss: 0.0425 - accuracy: 0.9847 - val_loss: 0.0328 - val_accuracy: 0.9853
Epoch 2/100
2969/2969 [==============================] - 38s 13ms/step - loss: 0.0210 - accuracy: 0.9930 - val_loss: 0.0610 - val_accuracy: 0.9783
Epoch 3/100
2969/2969 [==============================] - 37s 12ms/step - loss: 0.0129 - accuracy: 0.9960 - val_loss: 0.0820 - val_accuracy: 0.9756
Epoch 4/100
2969/2969 [==============================] - 37s 12ms/step - loss: 0.0091 - accuracy: 0.9969 - val_loss: 0.0572 - val_accuracy: 0.9850
Epoch 5/100
2969/2969 [==============================] - 37s 13ms/step - loss: 0.0083 - accuracy: 0.9974 - val_loss: 0.0636 - val_accuracy: 0.9837
Epoch 6/100
2969/2969 [==============================] - 37s 12ms/step - loss: 0.0066 - accuracy: 0.9979 - val_loss: 0.1015 - val_accuracy: 0.9781
Epoch 7/100
2969/2969 [==============================] - 37s 12ms/step - loss: 0.0055 - accuracy: 0.9981 - val_loss: 0

KeyboardInterrupt: ignored

In [13]:
pred_test = model.predict(test['content'].values)
pred_test

array([[2.8640344e-03],
       [2.1728119e-11],
       [3.2741721e-06],
       ...,
       [9.9996853e-01],
       [9.9999809e-01],
       [9.9999809e-01]], dtype=float32)

In [14]:
submission.loc[:, 'info'] = np.where(pred_test > 0.5, 1, 0).reshape(-1)

In [15]:
submission.loc[:, ['id', 'info']].to_csv(PATH + "v1.0.1.csv", index=False)
submission

,id,info
0,NEWS00237_1,0
1,NEWS00237_2,0
2,NEWS00237_3,0
3,NEWS00237_4,0
4,NEWS00237_5,0
...,...,...
142560,NEWS09482_72,1
142561,NEWS09482_73,1
142562,NEWS09482_74,1
142563,NEWS09482_75,1
